In [2]:
import news

In [3]:
feeds = news.GoogleNews()

Data successfully loaded.


In [6]:
feeds.get_titles()[:10]

['What Makes Alphabet (GOOG) a Good Long-Term Investment? - Yahoo Finance',
 'Why do Long-Term Investors Love Alphabet (GOOG)? - Yahoo Finance',
 '3 No-Brainer Stocks to Buy Right Now for Less Than $200 - The Motley Fool',
 'My Top Stock to Buy in a New Bull Market - The Motley Fool',
 'Google: Not Hyped, But Not Cheap Either (NASDAQ:GOOG) - Seeking Alpha',
 'BIDU vs. GOOG: Which Stock Should Value Investors Buy Now? - Yahoo Finance',
 "Why This YouTuber Isn't Buying What MrBeast Is Selling? - Alphabet (NASDAQ:GOOG), Alphabet (NASDAQ:GOOGL) - Benzinga",
 'Google: Why I See New Highs Coming (Rating Upgrade) (NASDAQ:GOOG) - Seeking Alpha',
 'Google Accused of Breaching Promised Ad Standards, Big Brands and Government Agencies Affected - Alphabet - Benzinga',
 '2 Cheap AI Stocks to Buy Right Now - The Motley Fool']